In [63]:
#import libs
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import time
import psutil


#loading face images
mat_content = sio.loadmat('face.mat')
face_data = mat_content['X']
face_labels = mat_content['l']
N = face_data.shape[1]

print(face_data.shape)
print(face_labels.shape)


(2576, 520)
(1, 520)


In [64]:
#partition training data
N_train = 8 #choose from 1 to 10
index = np.arange(520)
for x in range (N_train,10):
  index = index[(index-x) % 10 != 0] #removing 10-N_train images for each face
face_data_train = face_data[:,index]
print('training data dimension: ', face_data_train.shape)

#partition testing data
index = np.arange(520)
for x in range (N_train):
  index = index[(index-x) % 10 != 0] #removing N_train images for each face
face_data_test = face_data[:,index]
index = np.arange(520)
for x in range (N_train):
  index = index[(index-x) % 10 != 0] #removing N_train images for each face
face_labels_test = face_labels[:,index]
print('test data dimension: ', face_data_test.shape)
psutil.virtual_memory()


training data dimension:  (2576, 416)
test data dimension:  (2576, 104)


svmem(total=13655236608, available=12719828992, percent=6.9, used=1084141568, free=10706894848, active=814686208, inactive=1889181696, buffers=77463552, cached=1786736640, shared=864256, slab=156168192)

In [65]:
#PCA Mean 
N = face_data_train.shape[1]
print(N)
pca_mean = np.sum(face_data_train, axis=1)/N
pca_mean = np.reshape(pca_mean,(2576,1))
print(pca_mean.shape)

#LDA Mean
temp_list = np.zeros((2576,8))
lda_mean = np.zeros((2576,52))
for x in range (52):
  for y in range (8):
    temp_list[:,y] = face_data_train[:,x*8+y]
  lda_mean[:,x] = np.mean(temp_list, axis = 1)
print(lda_mean.shape)
print(np.mean(lda_mean, axis = 1))
print(lda_mean)
print(pca_mean)


416
(2576, 1)
(2576, 52)
[122.72355769 122.24278846 120.34134615 ...  84.81490385  82.81009615
  81.23798077]
[[119.125 134.25   93.625 ... 117.25  136.875 125.375]
 [119.5   140.125  98.75  ... 115.5   138.75  122.75 ]
 [120.5   136.5   101.375 ... 111.25  137.875 124.25 ]
 ...
 [ 58.625  59.875  93.375 ...  97.875  62.625  80.625]
 [ 58.5    61.     90.5   ...  95.     58.     84.625]
 [ 59.     57.375  71.    ...  92.375  58.875  93.   ]]
[[122.72355769]
 [122.24278846]
 [120.34134615]
 ...
 [ 84.81490385]
 [ 82.81009615]
 [ 81.23798077]]


In [66]:
#Compute S_i
S_w = np.zeros((2576,2576))
S_b = np.zeros((2576,2576))

for x in range (52):
  S_w_element = np.dot(8,np.dot((face_data_train[:,x*8:(x+1)*8] -np.reshape(lda_mean[:,x], (2576,1))),(face_data_train[:,x*8:(x+1)*8] -np.reshape(lda_mean[:,x], (2576,1))).T))
  S_w = S_w + S_w_element
print(S_w.shape)
print(S_w)

#Compute S_b
for x in range (52):
  S_b_element = np.dot((np.reshape(lda_mean[:,x], (2576,1))-pca_mean),(np.reshape(lda_mean[:,x], (2576,1))-pca_mean).T)
  S_b = S_b + S_b_element
print(S_b.shape)
print(S_b)



(2576, 2576)
[[ 6864091.  6476473.  5855970. ... -1544299. -1751929. -1729794.]
 [ 6476473.  6840419.  6415993. ... -1374999. -1581161. -1545714.]
 [ 5855970.  6415993.  6820030. ... -1358002. -1524741. -1450108.]
 ...
 [-1544299. -1374999. -1358002. ...  7424775.  7125520.  6737711.]
 [-1751929. -1581161. -1524741. ...  7125520.  7463611.  7139020.]
 [-1729794. -1545714. -1450108. ...  6737711.  7139020.  7509323.]]
(2576, 2576)
[[12217.97926683 11150.22445913 10302.00060096 ... -1509.23888221
  -1646.21424279 -1307.17277644]
 [11150.22445913 12143.01292067 12078.79987981 ... -2907.42878606
  -3155.33683894 -3096.47325721]
 [10302.00060096 12078.79987981 13810.22235577 ... -3539.93329327
  -3828.92608173 -4015.78665865]
 ...
 [-1509.23888221 -2907.42878606 -3539.93329327 ... 28148.98407452
  27067.04717548 25738.6811899 ]
 [-1646.21424279 -3155.33683894 -3828.92608173 ... 27067.04717548
  27864.32782452 27403.5375601 ]
 [-1307.17277644 -3096.47325721 -4015.78665865 ... 25738.6811899
 

In [67]:
#compute covariance matrix S = (1/N)A^T*A (w_PCA)
S_ld = np.dot((face_data_train - pca_mean).T, (face_data_train - pca_mean))/N
eigvals_S_ld, eigvecs_S_ld = np.linalg.eig(S_ld)

eigvecs_S_ld = np.dot(face_data_train - pca_mean ,eigvecs_S_ld)
eigvecs_S_ld = eigvecs_S_ld/np.linalg.norm(eigvecs_S_ld,axis=0, ord=2)

M_pca = 300;
W_pca = eigvecs_S_ld [:,:M_pca]

#W_pca = np.dot(face_data_train - pca_mean ,W_pca)
#W_pca = W_pca/np.linalg.norm(W_pca,axis=0, ord=2)

print(W_pca.shape)
print(W_pca)

#Compute w_LDA
S_b_modified = np.dot(np.dot(W_pca.T,S_b),W_pca)
S_w_modified = np.dot(np.dot(W_pca.T,S_w),W_pca)

print(S_b_modified.shape)
print(S_b_modified)
print(S_w_modified.shape)
print(S_w_modified)
print(np.linalg.inv(S_w_modified).shape)
print(np.linalg.inv(S_w_modified))

(2576, 300)
[[-0.01228713 -0.0192394   0.01088869 ...  0.00960605  0.03861788
  -0.02208343]
 [-0.01349973 -0.01952942  0.01215637 ... -0.00885586  0.03148095
   0.00384172]
 [-0.01402377 -0.02006991  0.01382132 ...  0.00495742 -0.01858449
   0.03757697]
 ...
 [ 0.0029651   0.04474409  0.02270606 ...  0.03669198 -0.0079936
   0.01646657]
 [ 0.00260247  0.04445823  0.02206885 ... -0.00719258 -0.0199451
   0.03740473]
 [ 0.00274912  0.04521234  0.02091973 ... -0.0299179  -0.02852776
  -0.00711917]]
(300, 300)
[[ 2.65933030e+07 -1.08591220e+06  6.63197138e+05 ... -1.90833505e+03
  -2.89234936e+04 -1.15067626e+04]
 [-1.08591220e+06  7.21224222e+06  2.49886774e+06 ... -2.61674828e+04
   8.25624317e+03  1.84345512e+04]
 [ 6.63197138e+05  2.49886774e+06  1.10186051e+07 ...  3.04892635e+03
   1.68038583e+04  1.66387389e+04]
 ...
 [-1.90833505e+03 -2.61674828e+04  3.04892635e+03 ...  2.82435143e+03
  -1.12010115e+02 -4.62935173e+02]
 [-2.89234936e+04  8.25624317e+03  1.68038583e+04 ... -1.12010

In [68]:
matrix = np.dot(np.linalg.inv(S_w_modified),S_b_modified)
eigvals_matrix, W_lda_full = np.linalg.eig(matrix)

M_lda = 50
W_lda = W_lda_full[:,:M_lda]

W_opt = (np.dot(W_lda.T, W_pca.T)).T
print(W_opt.shape)
print(W_opt.imag)
print(W_lda.shape)
print(W_lda)
#W_opt = W_opt/np.linalg.norm(W_opt,axis=0, ord=2)

(2576, 50)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(300, 50)
[[-0.02175485+0.j -0.00754926+0.j -0.00567115+0.j ...  0.00087703+0.j
  -0.00332575+0.j  0.00433014+0.j]
 [-0.00083728+0.j -0.01443246+0.j -0.0009017 +0.j ... -0.0004926 +0.j
   0.00743321+0.j -0.00454099+0.j]
 [ 0.01017269+0.j -0.00246084+0.j  0.02108573+0.j ...  0.00223928+0.j
  -0.00382641+0.j  0.00300589+0.j]
 ...
 [-0.15315885+0.j -0.00548216+0.j  0.06719492+0.j ... -0.01987035+0.j
   0.01009985+0.j  0.06045884+0.j]
 [-0.03560139+0.j -0.05012465+0.j  0.08806312+0.j ...  0.0371869 +0.j
   0.01663908+0.j -0.06013354+0.j]
 [ 0.08546921+0.j  0.04411019+0.j  0.06528773+0.j ... -0.06538804+0.j
   0.03560958+0.j -0.05460303+0.j]]


In [69]:
print(matrix[:6,:6])

[[ 0.36411826  0.00799131 -0.01443077  0.12047595 -0.06782977  0.01442982]
 [ 0.0315704   0.12971064  0.04580371  0.0941214  -0.02748242 -0.0286273 ]
 [-0.02541343  0.03710275  0.21542189  0.05712206  0.0774924   0.01684631]
 [ 0.45412231  0.154521    0.11896397  0.49600724 -0.02701682  0.00443416]
 [-0.33331699 -0.04207525  0.15082407 -0.04002443  0.3533894  -0.07799226]
 [ 0.11068074 -0.11360268  0.0643763   0.00691132 -0.1085231   0.17691462]]


In [70]:
#Nearest Neighbour for S=(1/N)AA^T
eigvecs = W_opt
print(eigvecs.shape)
face_coes_train = np.dot((face_data_train - pca_mean).T,eigvecs)
#face_coes_train = face_coes_train/np.linalg.norm(face_coes_train,axis=0, ord=2)

face_coes_test = np.dot((face_data_test - pca_mean).T,eigvecs) 
#face_coes_test = face_coes_test/np.linalg.norm(face_coes_test,axis=0, ord=2)

print(face_data_train - pca_mean)
print(face_coes_test.shape)
print(face_coes_train.shape)

classified_label = []
for n in range (face_coes_test.shape[0]):
  #difference = face_coes_train - face_coes_test[n,:]
  #neighbours_distance_squared = np.multiply(difference, difference)
  neighbours_distance_squared = np.multiply(face_coes_train - face_coes_test[n,:], face_coes_train - face_coes_test[n,:])
  neighbours_distance = np.sum(neighbours_distance_squared, axis = 1) #sum along rows
  nearest_index = np.argmin(neighbours_distance)
  nearest_index = 1+int(nearest_index/8)
  classified_label.append(nearest_index)
#print('classified label',classified_label)

face_labels_test = np.array(face_labels_test, dtype= np.int)
face_labels_test = np.reshape(face_labels_test, ((10-N_train)*52,))
#print(face_labels_test)
accuracy = face_labels_test - classified_label
print(accuracy)
non_zero_count = np.count_nonzero(accuracy)
accuracy = 100*(face_coes_test.shape[0] - non_zero_count)/face_coes_test.shape[0]
print('lda accuracy:',accuracy,'%')


(2576, 50)
[[-16.72355769   7.27644231  68.27644231 ...  58.27644231 -62.72355769
  -52.72355769]
 [-19.24278846   6.75721154  62.75721154 ...  47.75721154 -54.24278846
  -60.24278846]
 [-17.34134615   9.65865385  53.65865385 ...  42.65865385 -49.34134615
  -37.34134615]
 ...
 [-47.81490385 -44.81490385 -62.81490385 ... -23.81490385 -43.81490385
    2.18509615]
 [-46.81009615 -46.81009615 -63.81009615 ... -32.81009615 -34.81009615
    6.18990385]
 [-40.23798077 -46.23798077 -64.23798077 ... -36.23798077 -16.23798077
   12.76201923]]
(104, 50)
(416, 50)
[-37 -37   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 -29   0   0  -4  -4   1   0   0   0   0   0   0   0  13  13   0 -20
  15   0   0   0   0   0   0 -16   0   0   0   0   0   0   0   0   0   0
   0   0   0  -2  -8   0   0   0   0   0   0  -5   0   0   0  15   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   1   0  28   0   0   0
   0   0   0   0   0   0   0   0   0  12   0   0  33  14]
lda accuracy: 80.7

**Q3 part2**

In [71]:
eigvecs_S_ld.shape

(2576, 416)

In [72]:
M0 = 150
ro = 0.5
M1 = int((416 - 100)*ro)
T = 10
W_pca_rand_list = np.zeros((T,eigvecs_S_ld.shape[0],M1+M0))
np.random.seed(123) #fix seed


for t in range (T):
  rand_index = np.arange(416)[M0:]
  np.random.shuffle(rand_index)
  rand_index = rand_index[:M1]
  #generate W[M1:]
  W_rand = eigvecs_S_ld[:,rand_index]
  #combine M0 and M1
  W_pca_rand = np.concatenate((eigvecs_S_ld[:,:M0], W_rand), axis = 1)
  W_pca_rand_list[t,:,:] = W_pca_rand

print(W_pca_rand_list.shape)


(10, 2576, 308)


In [73]:
M_lda = 52
W_lda = W_lda_full[:,:M_lda]
W_opt_list = np.zeros((T,eigvecs_S_ld.shape[0],M_lda))

for t in range (T):
  S_b_modified = np.dot(np.dot(W_pca_rand_list[t,:,:].T,S_b),W_pca_rand_list[t,:,:])
  S_w_modified = np.dot(np.dot(W_pca_rand_list[t,:,:].T,S_w),W_pca_rand_list[t,:,:])

  matrix = np.dot(np.linalg.inv(S_w_modified),S_b_modified)
  eigvals_matrix, W_lda_full = np.linalg.eig(matrix)

  W_lda = W_lda_full[:,:M_lda]
  W_opt = (np.dot(W_lda.T, W_pca_rand_list[t,:,:].T)).T
  W_opt_list[t,:,:] = W_opt

W_opt_list.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  


(10, 2576, 52)

In [74]:
#average accuracy
ave_accuracy_list = np.zeros((T))
for t in range (T):
#Nearest Neighbour for S=(1/N)AA^T
  eigvecs = W_opt_list[t,:,:]
  face_coes_train = np.dot((face_data_train - pca_mean).T,eigvecs)
  face_coes_test = np.dot((face_data_test - pca_mean).T,eigvecs) 
  classified_label = []
  for n in range (face_coes_test.shape[0]):
    neighbours_distance_squared = np.multiply(face_coes_train - face_coes_test[n,:], face_coes_train - face_coes_test[n,:])
    neighbours_distance = np.sum(neighbours_distance_squared, axis = 1) #sum along rows
    nearest_index = np.argmin(neighbours_distance)
    nearest_index = 1+int(nearest_index/8)
    classified_label.append(nearest_index)

  face_labels_test = np.array(face_labels_test, dtype= np.int)
  face_labels_test = np.reshape(face_labels_test, ((10-N_train)*52,))
  accuracy = face_labels_test - classified_label
  non_zero_count = np.count_nonzero(accuracy)
  accuracy = 100*(face_coes_test.shape[0] - non_zero_count)/face_coes_test.shape[0]
  ave_accuracy_list[t] = accuracy
print(ave_accuracy_list)
ave_accuracy = np.mean(ave_accuracy_list)
print('average accuracy: ', ave_accuracy, '%')


[81.73076923 85.57692308 81.73076923 85.57692308 84.61538462 86.53846154
 81.73076923 86.53846154 80.76923077 84.61538462]
average accuracy:  83.94230769230771 %


In [75]:
from scipy import stats

#majority vote
classified_label_list = np.zeros((T,104))
for t in range (T):
#Nearest Neighbour for S=(1/N)AA^T
  eigvecs = W_opt_list[t,:,:]
  face_coes_train = np.dot((face_data_train - pca_mean).T,eigvecs)
  face_coes_test = np.dot((face_data_test - pca_mean).T,eigvecs) 
  classified_label = []
  for n in range (face_coes_test.shape[0]):
    neighbours_distance_squared = np.multiply(face_coes_train - face_coes_test[n,:], face_coes_train - face_coes_test[n,:])
    neighbours_distance = np.sum(neighbours_distance_squared, axis = 1) #sum along rows
    nearest_index = np.argmin(neighbours_distance)
    nearest_index = 1+int(nearest_index/8)
    classified_label.append(nearest_index)
  classified_label_list[t,:] = classified_label

classified_label_list.shape
#print(classified_label_list[:,0])
classified_label = stats.mode(classified_label_list)[0]
classified_label = np.reshape(classified_label, (104))
print(classified_label.shape)
face_labels_test = np.array(face_labels_test, dtype= np.int)
face_labels_test = np.reshape(face_labels_test, ((10-N_train)*52,))
accuracy = face_labels_test - classified_label
non_zero_count = np.count_nonzero(accuracy)
accuracy = 100*(face_coes_test.shape[0] - non_zero_count)/face_coes_test.shape[0]
print('majority vote accuracy: ',accuracy)

(104,)
majority vote accuracy:  87.5


In [76]:
#sum
face_coes_train_list = np.zeros((T,416,M_lda))
face_coes_test_list = np.zeros((T,104,M_lda))

for t in range (T):
#Nearest Neighbour for S=(1/N)AA^T
  eigvecs = W_opt_list[t,:,:]
  face_coes_train = np.dot((face_data_train - pca_mean).T,eigvecs)
  face_coes_test = np.dot((face_data_test - pca_mean).T,eigvecs) 
  face_coes_train_list[t,:,:] = face_coes_train
  face_coes_test_list[t,:,:] = face_coes_test

face_coes_train = np.mean(face_coes_train_list,axis = 0)
face_coes_test = np.mean(face_coes_test_list,axis = 0)

#print(np.mean(face_coes_train_list[:,0,0]))
#print(face_coes_train[0,0])

classified_label = []
for n in range (face_coes_test.shape[0]):
  #difference = face_coes_train - face_coes_test[n,:]
  #neighbours_distance_squared = np.multiply(difference, difference)
  neighbours_distance_squared = np.multiply(face_coes_train - face_coes_test[n,:], face_coes_train - face_coes_test[n,:])
  neighbours_distance = np.sum(neighbours_distance_squared, axis = 1) #sum along rows
  nearest_index = np.argmin(neighbours_distance)
  nearest_index = 1+int(nearest_index/8)
  classified_label.append(nearest_index)
#print('classified label',classified_label)

face_labels_test = np.array(face_labels_test, dtype= np.int)
face_labels_test = np.reshape(face_labels_test, ((10-N_train)*52,))
#print(face_labels_test)
accuracy = face_labels_test - classified_label
print(accuracy)
non_zero_count = np.count_nonzero(accuracy)
accuracy = 100*(face_coes_test.shape[0] - non_zero_count)/face_coes_test.shape[0]
print('sum accuracy: ',accuracy,'%')

[  0 -37   0   0   0   0   0   0   0   0   0   0   0   0  -6   0   0   0
   0 -12   0   6  -4 -18  -3   0   0   0 -30   0   0   0   1  -7   0   0
  -1   0   0   0   0   0   3   2   0   0   0   0   0   0 -20  24   0   0
   0   0   0   0  25   1   0   0   0   0   0   4   5   5   0  -6   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   1   0  28   0   0  37
   0   0   0  18   0   0   0   0   0  12   0   0   0   9]
sum accuracy:  74.03846153846153 %
